In [7]:
# imports and loading
from pymodulon.io import *
from pymodulon.plotting import *
from scipy.stats import fisher_exact, chi2_contingency
from statsmodels.stats.multitest import multipletests
import os
from os import path
import pandas as pd
import seaborn as sns
import ast 
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

base_dir = '/home/chris/zuniga/iModulonMiner'
external_data = path.join(base_dir,'data','external')
df_trn = pd.read_csv(path.join(external_data,'TRN.csv'))

data_dir = path.join(base_dir,'data','processed_data')
interim_dir = path.join(base_dir,'data','interim')
ica_data = load_json_model(path.join(data_dir,'ZM4_ica.json.gz'))
ica_data.gene_table.to_csv(path.join(data_dir, 'supp_gene_table.csv'))

/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(table)
/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(table)
/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_json(table)
/home/chris/miniforge3/envs/iM_miner/lib/python3.10/site-packages/pymodulon/util.py:31: FutureWarning: Passing literal json to 'read_json' 

In [8]:
# add to iM_table, print out for supplement
df = ica_data.imodulon_table.copy()
to_add = []
for index, row in df.iterrows():
    genes = list(ica_data.view_imodulon(index).index)
    genes.sort()
    to_add.append(genes)
df['genes'] = to_add
df.to_csv(path.join(data_dir, 'supp_iM_table.csv'))

In [14]:

# let's create groups of regulators

# first, by iModulon
iM_to_genes = {}
for iM in ica_data.imodulon_table.index:
    iM_to_genes.update({iM : list(ica_data.view_imodulon(iM).sort_index().index)})
pd.Series(iM_to_genes).to_csv(path.join(data_dir, 'supp_iM_to_genes.csv'))

# second by DAP-seq
DAP_to_genes = {}
for reg, df in df_trn.groupby('regulator'):
    DAP_to_genes.update({reg : set(df['gene_id'])})
for k, v in DAP_to_genes.items():
    new = list(v)
    new.sort()
    DAP_to_genes.update({k : new})
pd.Series(DAP_to_genes).to_csv(path.join(data_dir, 'supp_DAP_to_genes.csv'))

# third by seq-based
tot_regs = set(ica_data.gene_table.FIMO_regulators)
all_regs = set()
for regs in tot_regs:
    if str(regs) == 'nan': continue
    for reg in ast.literal_eval(regs):
        all_regs.add(reg)
FIMO_to_genes = {reg : set() for reg in all_regs}
for index, row in ica_data.gene_table.iterrows():
    if str(row['FIMO_regulators']) == 'nan': continue
    for reg in ast.literal_eval(row['FIMO_regulators']):
        FIMO_to_genes[reg].add(index)
for k, v in FIMO_to_genes.items():
    new = list(v)
    new.sort()
    FIMO_to_genes.update({k : new})
pd.Series(FIMO_to_genes).to_csv(path.join(data_dir, 'supp_FIMO_to_genes.csv'))

In [11]:
regs

nan